# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books(min_rating, max_price):

    domain = "https://books.toscrape.com/catalogue/"
    page_url = domain + "page-1.html"
    book_list = []
    rating_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    while True:

        response = requests.get(page_url) # We send a request to the page URL
        soup = BeautifulSoup(response.content, "html.parser") # We parse the content of the page with BeautifulSoup

        products = soup.find_all("article", class_="product_pod")

        for product in products:

            # Rating
            rating_class = product.find("p")["class"] # We retrieve the class list in order to get the rating
            book_rating = rating_dict.get(rating_class[1], 0) # We convert the rating from string to integer by using the dictionary

            # Price
            price = product.find("p", class_="price_color").get_text() # We retrieve the price string
            price_clean = float(price.replace("£", "")) # We clean the price string and convert it to float

            # Then we use an if statement to filter books based on rating and price
            if book_rating >= min_rating and price_clean <= max_price:

                # Title and book URL
                title = product.h3.a["title"] # We retrieve the book title
                partial_url = product.h3.a["href"] # We retrieve the partial URL of the book
                book_url = domain + partial_url # We create the full URL of the book by concatenating the domain and the partial URL

                # Book details
                book_response = requests.get(book_url) # We send a request to the book URL
                book_soup = BeautifulSoup(book_response.content, "html.parser") # We parse the content of the book page with BeautifulSoup
                
                # UPC, Genre, Availability
                upc = book_soup.find("th", string="UPC").find_next("td").get_text() # We retrieve the UPC
                genre = book_soup.find("ul", class_="breadcrumb").find_all("li")[2].get_text(strip=True) # We retrieve the genre
                availability = book_soup.find("p", class_="instock availability").get_text(strip=True) # We retrieve the availability

                # Description
                desc_tag = book_soup.find("div", id="product_description") # We need to check if the description exists by looking for the tag first
                if desc_tag:
                    description = desc_tag.find_next("p").get_text() # If it exists, we retrieve the description
                else:
                    description = "No description available."
                
                # We compile all the information into a dictionary with the name of the columns we want in our DataFrame
                book_infos = {
                    "UPC": upc,
                    "Title": title,
                    "Price (£)": price,
                    "Rating": book_rating,
                    "Genre": genre,
                    "Availability": availability,
                    "Description": description
                }

                book_list.append(book_infos) # We add the dictionary to the list

        # We check if there is a next page
        next_btn = soup.find("li", class_="next")
        if next_btn:
            next_page = next_btn.a["href"]
            page_url = domain + next_page # We update the page URL to the next page by concatenating the domain and the next page URL
        else:
            break

    return pd.DataFrame(book_list) # We convert the list of dictionaries to a DataFrame and return it


df_books = scrape_books(min_rating=4, max_price=20.00)
print(f"{len(df_books)} books found with a rating of at least 4 stars and a price less than £20!")
df_books.head()

# The most difficult challenge in this exercice for me was first to understand why I was getting only one book in my final DataFrame.
# I realized after a long time that I was checking only the first page of the website and not iterating through all the pages.
# Then, I had to figure out how to navigate to the next page by looking for the "next" button and updating the page URL accordingly.
# Also, I was getting errors when trying to retrieve the book description because some books did not have a description.
# I had to add a check to see if the description tag existed before trying to retrieve the description text.
# Compare to what we did in class, I found web scraping pretty challenging cause it's not really intuitive at first.


75 books found with a rating of at least 4 stars and a price less than £20!


,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,£17.46,5,Young Adult,In stock (19 available),Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,£17.66,5,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,£15.94,5,Philosophy,In stock (18 available),A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,£14.27,4,Poetry,In stock (16 available),"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,£19.49,4,Sequential Art,In stock (16 available),"Every summer, Rose goes with her mom and dad t..."
